In [1]:
import pandas as pd
import json
from my_jira import my_jira_reports

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('data.json', 'r') as read_file:
    data = json.load(read_file)
jr=my_jira_reports(data['server'],data['auth'],verify_cert=False)
project='MIGRCEH3'

### Отчёт по незавершенным задачам

In [8]:
df=jr.undone_custom_report(project)
jr.color_undone_custom_report(df)

,key,assignee,update_state,status,start_date,end_date,sprint_state,sprint_name,term
0,MIGRCEH3-151,Громов Иван Андреевич (4061128),ok,Создано,2022-08-01,2022-08-03,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,2d/2d
1,MIGRCEH3-171,Исхакова Эидис Алексеевна,ok,Создано,2022-08-02,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,1d/1d
2,MIGRCEH3-164,Фиданян Геворк Саркисович,ok,Создано,2022-08-01,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,2d/2d
3,MIGRCEH3-160,Волочков Илья Сергеевич,ok,Создано,2022-08-01,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,2d/2d
4,MIGRCEH3-156,Якубовский Ян Олегович (4061619),ok,Создано,2022-08-02,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,1d/1d
5,MIGRCEH3-147,Громов Иван Андреевич (4061128),ok,В работе,2022-07-21,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,2d/2d
6,MIGRCEH3-136,Чернышев Павел Алексеевич,ok,Создано,2022-08-01,2022-08-02,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,2d/2d
7,MIGRCEH3-170,Исхакова Эидис Алексеевна,ok,Создано,2022-07-28,2022-08-01,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,3d/3d
8,MIGRCEH3-155,Якубовский Ян Олегович (4061619),ok,Создано,2022-08-01,2022-08-01,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,1d/1d
9,MIGRCEH3-137,Чернышев Павел Алексеевич,ok,Создано,2022-07-28,2022-08-01,ACTIVE,Спринт 2022.3.2.NPP Shared Sprint,3d/3d


### Обновляем статусы

In [ ]:
#jr.get_task_transitions('MIGRCEH3-127')
#jr.transition_issue('MIGRCEH3-127',121,'Готово')
jr.mass_transition(df)

In [ ]:
df=jr.undone_custom_report(project)
jr.color_undone_custom_report(df)

### Инфо по задаче

In [4]:
full_info=jr.get_issue('MIGRCEH1-642')
jr.extract_issue_info(full_info)

{'key': 'MIGRCEH1-642',
 'assignee': 'Волочков Илья Сергеевич',
 'parent': None,
 'resolution': None,
 'issuelinks': [{'id': '140859498',
   'self': 'https://task.corp.dev.vtb/rest/api/2/issueLink/140859498',
   'type': {'id': '10811',
    'name': 'Depend',
    'inward': 'is required by',
    'outward': 'depend on',
    'self': 'https://task.corp.dev.vtb/rest/api/2/issueLinkType/10811'},
   'outwardIssue': {'id': '140391384',
    'key': 'MIGRCEH1-577',
    'self': 'https://task.corp.dev.vtb/rest/api/2/issue/140391384',
    'fields': {'summary': 'dm_payments_report витрина отчётного слоя по платежам',
     'status': {'self': 'https://task.corp.dev.vtb/rest/api/2/status/11200',
      'description': '',
      'iconUrl': 'https://task.corp.dev.vtb/images/icons/statuses/generic.png',
      'name': 'Создано',
      'id': '11200',
      'statusCategory': {'self': 'https://task.corp.dev.vtb/rest/api/2/statuscategory/2',
       'id': 2,
       'key': 'new',
       'colorName': 'blue-gray',
    

In [ ]:
full_info['fields']['issuelinks']

### Команда

In [ ]:
jr.show_team(data['team'])

### Worklog

In [ ]:
%%time
start_date='2021-10-01'
wl=jr.worklog_rep(data['team'],start_date=start_date)
wr=wl[wl.log_date>=start_date]

In [ ]:
wr.pivot_table(index=['key','summary'], columns='author', values='hours',aggfunc=sum,margins=True).fillna(0)

### Создание задач

In [ ]:
_path=r'Z:\планирование VTB\2022_3_2.xlsx'
input_data=pd.read_excel(_path,parse_dates=False)
input_data.assignee.unique()
team_dict={'Громов':'VTB4061128',
          'Якубовский':'VTB4061619','Волочков':'VTB70173009@region.vtb.ru',
           'Фиданян':'VTB70172953@region.vtb.ru',
           'Исхакова':'VTB70172998',
           'Чернышев':'VTB70172956'
          }
input_data['assignee_name']=input_data['assignee'].apply(lambda x:team_dict[x])
input_data

In [ ]:
def vtb_issue(project:str,row:dict,sprint=None):
    additional_keys={'customfield_16206': [{'key':'ST-191257'}],
                 'customfield_16201': [{'key':'SYS-358683'}],
                 'customfield_15715': [{'key':'AG-1453383'}],
                     'customfield_11164':str(row['start_date'])[:10],
                     'customfield_11163':str(row['end_date'])[:10],
                     'duedate':str(row['end_date'])[:10],
                 'priority':{'name': 'Medium'},
                 "timetracking": {"originalEstimate": row['estimate'],
                                  "remainingEstimate": row['estimate'],
                                 },
                 "assignee":{'name': row['assignee_name']}
                }
    issue=jr.create_issue(project=project,
                  issue_type='Задача',
                  task_summary=row['task_summary'],
                  task_description='',
                  additional_keys=additional_keys)
    epic=row['epic']
    #return issue,epic
    key=issue.json()['key']
    if sprint is not None:
        jr.issue_to_sprint(sprint, [key])
    
    if row['epic'] is not pd.np.nan:
        jr.issue_to_epic(row['epic'],[key])
        
    return (i,key,row['assignee'])
    

In [ ]:
jr.issue_to_epic('ANALYTREP-1328',['MIGRCEH3-148'])

In [ ]:
#jr.issue_to_sprint('158345', ['MIGRCEH3-{}'.format(el) for el in range(149,177) ])

In [ ]:
for i,row in input_data.iterrows():
    r=vtb_issue('MIGRCEH3',row,158345)

In [ ]:
#res=[]
for i,row in input_data.iterrows():
    if i>0:
        res.append(vtb_issue('MIGRCEH1',row))

In [ ]:
pd.DataFrame(res).to_e

In [ ]:
jr.issue_to_epic('ANALYTREP-1328',['MIGRCEH3-148'])

In [ ]:
additional_keys={'customfield_16206': [{'key':'ST-191257'}],
                 'customfield_16201': [{'key':'SYS-358683'}],
                 'customfield_15715': [{'key':'AG-1453383'}],
                 'priority':{'name': 'Medium'},
                 "timetracking": {"originalEstimate": "1w 3d",
                                  "remainingEstimate": "1w 3d",
                                 },
                 "assignee":{'name': 'VTB70173009@region.vtb.ru'}
                }

In [ ]:
t=jr.create_issue(project='MIGRCEH1',
                  issue_type='Задача',
                  task_summary='Корневая задача branch_x_abs_hist',
                  task_description='',
                  additional_keys=additional_keys)

In [ ]:
d=jr.issue_raw_info('MIGRCEH3-147')['fields']
{k: v for k, v in d.items() if v is not None}['customfield_15715']

In [ ]:
{"set": [{"key" : "TEST-1"}]}

In [ ]:
jr.update_issue('MIGRCEH3-157',{'customfield_15715':
                               [{"key":"546"}]
                               })

In [ ]:
d=jr.issue_raw_info('MIGRCEH3-157')['fields']
{k: v for k, v in d.items() if v is not None}['customfield_15715']